In [32]:
import cv2
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=True, device=device, thresholds=[0.5, 0.5, 0.5])
model = InceptionResnetV1(pretrained="casia-webface", classify=False).to(device)
model.eval()

In [ ]:
face_vectors = torch.load("./face_embeddings.pt") # a list of face embeddings
face_names = np.load("./face_names.npy") # a list of names corresponding to the embeddings

In [38]:
def cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1, vec2, dim=0).item()

image = cv2.imread("./dataset/test_image/Aaron_Guiel.jpg")
face_detected = mtcnn(image)

similarities = []

if face_detected is not None:
    feature_vector = model(face_detected.to(device))[0] # [1, 512]
    for i, face_vector in enumerate(face_vectors):
        sim = cosine_similarity(feature_vector, face_vector)
        if sim > 0.5:
            similarities.append((face_names[i], sim))
    if similarities:
        similarities.sort(key=lambda x: x[1], reverse=True)
        print(f"Top matches for the detected face:")
        for name, sim in similarities[:5]:
            print(f"Name: {name}, Similarity: {sim:.4f}")
else:
    print("No face detected in the image.")

Top matches for the detected face:
Name: Claire_Hentzen, Similarity: 0.5994
Name: Aaron_Guiel, Similarity: 0.5657
Name: Rick_Bland, Similarity: 0.5362
Name: Jake_Brace, Similarity: 0.5240
Name: Michael_Deutsch, Similarity: 0.5186
